In [1]:
import src
import pvlib
import numpy as np
import pandas as pd

# 1. Step 1. Upload input dataset

In [2]:
df = pd.read_csv(filepath_or_buffer=f'./validations/data/bogota-5.csv',
                 sep=',',
                 decimal='.',
                 index_col='timestamp',
                 parse_dates=True)

df.head()

,ghi_wm2,poa_wm2,tmod_c,tamb_c
timestamp,,,,
2020-01-01 00:00:00,0.0,1.022,14.682,27.484
2020-01-01 00:05:00,0.0,1.018,14.432,27.360
2020-01-01 00:10:00,0.0,1.014,13.980,26.912
2020-01-01 00:15:00,0.0,1.008,13.288,26.331
2020-01-01 00:20:00,0.0,1.000,12.772,25.794


## 1.1. Clear-sky index

If the clear-sky index ($k_c$) is not in the input dataset, it can be estimated with the function `src.methods.clear_sky_index` which requires the following input parameters: `data`, `column` (irradiance column to estimate $k_c$, e.g., `'ghi_wm2'` or `'poa_wm2'`), `longitude`, `latitude`, `altitude` and `time_zome` of the location.

In [3]:
# Location
longitude = -74.0674
latitude = 4.6024
altitude = 2624 # [m]
tz = 'America/Bogota'

In [4]:
df = src.methods.clear_sky_index(data=df,
                                 column='poa_wm2',
                                 longitude=longitude,
                                 latitude=latitude,
                                 altitude=altitude,
                                 time_zone=tz)

df.head()

,ghi_wm2,poa_wm2,tmod_c,tamb_c,ics_wm2,kc
timestamp,,,,,,
2020-01-01 00:00:00,0.0,1.022,14.682,27.484,0.0,1.0
2020-01-01 00:05:00,0.0,1.018,14.432,27.360,0.0,1.0
2020-01-01 00:10:00,0.0,1.014,13.980,26.912,0.0,1.0
2020-01-01 00:15:00,0.0,1.008,13.288,26.331,0.0,1.0
2020-01-01 00:20:00,0.0,1.000,12.772,25.794,0.0,1.0


# 2. Step 2. Define parameters

In [5]:
DF = df                 # Input dataset. Must be a pd.Dataframe.
COL = 'poa_wm2'         # Column of the pd.DataFrame of the input data.
YEAR = 2023             # Year of study. Must exist in the input dataset.
MONTH = 1               # Month of study (1 to 12). Must exist in the input dataset.
SC = 'sc5'              # Sky condition of study. Must be 'sc1', 'sc2', 'sc3', 'sc4' or 'sc5'.
METHOD = 'stochastic'   # Method to generate the synthetic data. Must be 'stochastic' or 'bootstrap'.
IC = 0.95               # Confidence interval. Must be a value between 0 and 1.
RUNS = 5                # Quantity of one-day synthetic sequences to generate.

# 3. Step 3. Execute the synthetic data generation method

## 3.1. Unique date execution

Use the following code to run the synthetic build method once for the parameters defined above.

In [6]:
synthetic = src.methods.sequential(data=DF,
                                   irradiance_column=COL,
                                   year=YEAR,
                                   month=MONTH,
                                   sky_condition=SC,
                                   method=METHOD,
                                   confidence_interval=IC,
                                   runs=RUNS)

synthetic.head()

synt1     synt2     synt3     synt4     synt5
hour minute                                                  
0    0       0.929305  0.935197  0.932224  0.933194  0.934838
     5       0.928000  0.938000  0.931458  0.928914  0.935369
     10      0.926000  0.932000  0.926000  0.929414  0.932000
     15      0.929874  0.930000  0.928982  0.930000  0.930000
     20      0.931793  0.928657  0.930742  0.932000  0.928348

## 3.2. Multiple execution

If you want to generate the synthetic data for the entire input data set, run the following code. It will generate a one-day `RUNS` solar irradiance sequence for all sky conditions for each month and year in the input data set.

In [7]:
SYNTHETIC = {'stochastic': {}, 'bootstrap': {}}

LOOP_YEARS = DF.index.year.unique().values
LOOP_MONTHS = DF.index.month.unique().values
LOOP_SC = ['sc1', 'sc2', 'sc3', 'sc4', 'sc5']

MONTHS = {'1': 'Jan', '2': 'Feb', '3': 'Mar', '4': 'Apr', '5': 'May', '6': 'Jun',
          '7': 'Jul', '8': 'Aug', '9': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'}

for i in LOOP_YEARS:
    for j in LOOP_MONTHS:
        for k in LOOP_SC:
            # Stochastic
            SYNTHETIC['stochastic'][f'{MONTHS[str(j)]}{i}-{k}'] = src.methods.sequential(data=DF,
                                                                                         irradiance_column=COL,
                                                                                         year=YEAR,
                                                                                         month=j,
                                                                                         sky_condition=k,
                                                                                         method='stochastic',
                                                                                         confidence_interval=IC,
                                                                                         runs=RUNS)
            # Bootstrap
            SYNTHETIC['bootstrap'][f'{MONTHS[str(j)]}{i}-{k}'] = src.methods.sequential(data=DF,
                                                                                        irradiance_column=COL,
                                                                                        year=YEAR,
                                                                                        month=j,
                                                                                        sky_condition=k,
                                                                                        method='bootstrap',
                                                                                        confidence_interval=IC,
                                                                                        runs=RUNS)

SYNTHETIC

{'stochastic': {'Jan2020-sc1':                 synt1     synt2     synt3     synt4     synt5
  hour minute                                                  
  0    0       0.288500  0.935287  1.191499  0.419836  1.620814
       5       0.000000  0.909791  0.908000  0.000000  1.620814
       10      0.000000  0.910000  0.910000  0.000000  1.620814
       15      0.000000  0.910000  0.910000  0.000000  1.620814
       20      0.000000  0.914000  0.910242  0.000000  1.620814
  ...               ...       ...       ...       ...       ...
  23   35      0.911490  0.952000  0.952000  0.910000  0.910000
       40      0.915667  0.952000  0.952000  0.914796  0.916000
       45      0.912428  0.956000  0.956000  0.913885  0.910000
       50      0.910000  0.952000  0.952000  0.910000  0.910000
       55      0.912000  0.948000  0.948000  0.909334  0.910471
  
  [288 rows x 5 columns],
  'Jan2020-sc2':                 synt1     synt2     synt3     synt4     synt5
  hour minute                  